# Storage

Pods and containers are __ephemeral__, which has it's upsides and downsides:

The upsides to pods and containers as we have seen:
- Easy to create
- Easy to destroy 
- Immutable and easier to reason about
- Easy to parallelize

But, unfortunately, there are some downsides:
- When a pod is removed, all of the data created by it on local disk is also removed
- Each container is separate entity, hence sharing data between containers is not possible...

... not possible unless we introduce `Volume`s!


# Volumes

`Volume`s in `k8s` are a little different from the ones in `Docker`.

Brief overview of `volume`s in `Docker`:
- Directory on `disk` or in another `container`
- `Volume`s are mounted to containers during runtime
- One can share data across instances (or using `cloud` storage) via `drivers` (see [here](https://docs.docker.com/storage/volumes/#share-data-among-machines))

Take into account that, even though this resolves some of our problems, the feature set is quite limited and is not enough for handling large-scale deployments.

High level features:
- __Any `volume`s can be mounted at the same time__
- __Ephemeral `volume`s__: Their lifetime is the same as `POD`
- __Persistent `volume`s__: They are independent of `POD` lifetime
- __Data is available across `container`s restart__ handled by `kubelet`

You can attach volumes to pods via:
- `.spec.volumes`: specifies which `volumes` to use
- `.spec.containers[*].volumeMounts`:  where and which volume to mount for aspecific container

Let see an example using a `DaemonSet`:

```
apiVersion: apps/v1
kind: DaemonSet
metadata:
  name: fluentd-elasticsearch
  labels:
    k8s-app: fluentd-logging
spec:
  selector:
    matchLabels:
      name: fluentd-elasticsearch
  template:
    metadata:
      labels:
        name: fluentd-elasticsearch
    spec:
      containers:
      - name: fluentd-elasticsearch
        image: quay.io/fluentd_elasticsearch/fluentd:v2.5.2
        resources:
          limits:
            memory: 200Mi
          requests:
            cpu: 100m
            memory: 200Mi
        # Here we can mount them with `name` matching
        # Ephemeral Volumes
        volumeMounts:
        - name: varlog
          mountPath: /var/log
        - name: varlibdockercontainers
          mountPath: /var/lib/docker/containers
          readOnly: true
      terminationGracePeriodSeconds: 30
      # Here we define our volumes
      # Data from POD will be mounted
      volumes:
      - name: varlog
        hostPath:
          path: /var/log
      - name: varlibdockercontainers
        hostPath:
          path: /var/lib/docker/container

## Try it out

1. Create a .yaml file for creating the DeamonSet
2. Create it and use the describe command from kubectl to observe the volumes
3. Go to the minikube dashboard and look for the volumes

In [1]:
# Create the DaemonSet
!kubectl

daemonset.apps/fluentd-elasticsearch created


In [4]:
# Use the describe command
!kubectl

Name:           fluentd-elasticsearch
Selector:       name=fluentd-elasticsearch
Node-Selector:  <none>
Labels:         k8s-app=fluentd-logging
Annotations:    deprecated.daemonset.template.generation: 1
Desired Number of Nodes Scheduled: 3
Current Number of Nodes Scheduled: 3
Number of Nodes Scheduled with Up-to-date Pods: 3
Number of Nodes Scheduled with Available Pods: 3
Number of Nodes Misscheduled: 0
Pods Status:  3 Running / 0 Waiting / 0 Succeeded / 0 Failed
Pod Template:
  Labels:  name=fluentd-elasticsearch
  Containers:
   fluentd-elasticsearch:
    Image:      quay.io/fluentd_elasticsearch/fluentd:v2.5.2
    Port:       <none>
    Host Port:  <none>
    Limits:
      memory:  200Mi
    Requests:
      cpu:        100m
      memory:     200Mi
    Environment:  <none>
    Mounts:
      /var/lib/docker/containers from varlibdockercontainers (ro)
      /var/log from varlog (rw)
  Volumes:
   varlog:
    Type:          HostPath (bare host directory volume)
    Path:          /var

The dashboard should look like this, with the volumes mounted to the container:
<p><img src=images/volume.png></p>

## Volume Types

`kubernetes` provides quite a few integrations for standard volumes, including:
- `AWS Elastic Block Store`
- Microsoft's Azure `Disk` and `File`
- Self-hosted `cephfs`
- [Google Cloud Persistent Disk](https://kubernetes.io/docs/concepts/storage/volumes/#gcepersistentdisk)

An example config with `awsElasticBlockStore` could be:

In [ ]:
apiVersion: v1
kind: Pod
metadata:
  name: test-ebs
spec:
  containers:
  - image: k8s.gcr.io/test-webserver
    name: test-container
    volumeMounts:
    - mountPath: /test-ebs
      name: test-volume
  volumes:
  - name: test-volume
    # This AWS EBS volume must already exist.
    awsElasticBlockStore:
      volumeID: "<volume id>"
      fsType: ext4

to see volume types in more detail [check here](https://kubernetes.io/docs/concepts/storage/volumes/#volume-types).

__These are all ephemeral `volume`s hence they will only live as long as its pod!__

# Persistent Volumes

> `Kubernetes` provides two resources to manage persistent storage: `PersistentVolume` and `PersistentVolumeClaim`

This abstraction allows us to:
- Abstract how storage is provided
- Abstract a way storage is consumed 

## PersistentVolume

A persistent volume is a piece of storage in a cluster that an administrator has provisioned. Persistent volumes are 'physical' volumes on the host machine

Persistent volumes have the following characteristics:

- resource in the cluster (just like `Node`)
- are volume plugins just like `Volume`s described above
- __they have lifecycle independent of any pod using it__
- when bounded can be used just like `volume`

## PersistentVolumeClaim

A persistent volume claim (PVC) is a request for the platform to create a PV for you. It will look for an 'Available' PersistentVolume that meets the specified requirements.

Features:
- Conceptually similiar to `POD`s:
    - `POD`s consume `Node` resources
    - `PVC`s consume `PV` resources
    - `POD`s can request specific resources (e.g. RAM memory)
    - `PVC`s can request specific sizes and access modes (e.g. `read` and `write`)
    
## Provisioning

There are two ways `PV` can be provisioned:
- `statically` - cluster admin creates `PV`s for consumption
- `dynamically` - cluster tries to dynamically provision appropriate `PersistentVolume` based on `PersistentVolumClaim`'s __`StorageClasses`__ (administrator has to provision `StorageClass`, __described later__)

Features:
- __`Dynamic Provision` will always match exactly the requirements of `PVC`__
- __`Static Provision` has to match AT LEAST the given claim__ (e.g. claim of `50Gb` might be given `100Gb`)

## Reclaim Policy

When a user is done with their volume, they can delete the PVC objects from the API that allows reclamation of the resource

There are three ways to reclaim the resource:
- [`retain`](https://kubernetes.io/docs/concepts/storage/persistent-volumes/#retain) - leave the data as is (leave `PersistentVolume` and 
- [`delete`](https://kubernetes.io/docs/concepts/storage/persistent-volumes/#delete) - delete `PersistentVolume` __and external storage__. This one is default for `Dynamic Provisioning` (although configurable)
- `recycle` (__now deprecated, dynamic provisioning should be used instead__, see [here](https://kubernetes.io/docs/concepts/storage/persistent-volumes/#recycle))

Volumes can show 4 states:

- `Available`: It means that this volume is ready to be bound to a pod
- `Bound`: CLI will show pod to which `PersistentVolume` is bound
- `Released`: `PesistentVolumeClaim` ended, but resource is not yet reclaimed by `cluster`
- `Failed` - Reclamation failed

## Specifying `PersistentVolume`

As with pod, we create them using `.yaml` config files, example below:

```
apiVersion: v1
kind: PersistentVolume
metadata:
  name: task-pv-volume
  labels:
    type: local
spec:
  storageClassName: manual
  capacity:
    storage: 10Gi
  accessModes:
    - ReadWriteOnce
  hostPath:
    path: "/mnt/data"
```

Let's describe some arguments you can provide:
1. `.spec.capacity.storage` - request `10Gb` of storage. Currently only storage can be requested (see [`k8s` resource model for description of units](https://github.com/kubernetes/community/blob/master/contributors/design-proposals/scheduling/resources.md))
2. `.spec.volumeMode` - either `FileSystem` (default) or `Block`:
    - `FileSystem` - directory mounted in pods
    - `Block` - uses raw block of storage (without filesystem created). It is `Block` is rarely used as application needs to know how to access `raw` data (see [here](https://kubernetes.io/docs/concepts/storage/persistent-volumes/#raw-block-volume-support) for example)
3. `.spec.accessModes` - how data can be accessed:
    - `ReadWriteOnce` - volume mounted as `rw` by a single `Node`
    - `ReadOnlyMany` - can be mounted by many `nodes` but data can only be read
    - `ReadWriteMany` - as above but both `rw` (applications have to handle possible data races!)
    - `ReadWriteOncePod` - single pod can `rw` data
      
Modes above differ by type of `PersistentVolume` provider, a few of them shown below:

![](./images/modes_providers.png)

4. __`.spec.storageclassName`__ - specifies `StorageClass`, if left unspecified __there is no `StorageClass` specified and only `PV` without one can be matched to `POD`!__
5. `.spec.mountOptions`- (__NOT SUPPORTED BY ALL TYPES!__); specifies how to mount `disk`, one can leave it as is

## Try it out

1. Create a .yaml file with the configuration above for creating a PersistentVolume resource
2. `apply` it using the right `kubectl`
3. Observe the status of the volume

In [8]:
!kubectl

persistentvolume/task-pv-volume created


In [11]:
!kubectl

NAME             CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS      CLAIM   STORAGECLASS   REASON   AGE
task-pv-volume   10Gi       RWO            Retain           Available           manual                  13m


# StorageClasses

> A StorageClass provides a way for administrators to describe the "classes" of storage they offer.

These allow us to __dynamically provision storage__ and acts like a template for new `PersistentVolume`.

As per usual, these are defined using `.yaml` files and can be referred by `PV` config files.

A small example:

In [ ]:
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: standard
provisioner: kubernetes.io/aws-ebs
parameters:
  type: gp2
reclaimPolicy: Retain
allowVolumeExpansion: true
mountOptions:
  - debug
volumeBindingMode: Immediate

> ### `.metadata.name` value allows users to request this `StorageClass`!

## Mandatory Fields

> `provisioner` - which volume plugin is used for provisiong `PV`

Most common ones are shipped with `k8s` under `kubernetes.io` prefix, for example:

- `local` - `kubernetes.io/no-provisioner` - create `PV`s dynamically from local resources

In [ ]:
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: local-storage
provisioner: kubernetes.io/no-provisioner
volumeBindingMode: WaitForFirstConsumer

- `GCEPersistentDisk` - Persistent disk from Google Cloud

In [ ]:
apiVersion: storage.k8s.io/v1
kind: StorageClass
metadata:
  name: slow
provisioner: kubernetes.io/gce-pd
parameters:
  type: pd-standard
  fstype: ext4
  replication-type: none

We are not limited to the ones provided, `provider`s can be written by anyone and hosted

> `parameters` define per-provisioner specification of volume properties

You can see examples for different internal provisioners [here](https://kubernetes.io/docs/concepts/storage/storage-classes/#aws-ebs)

> `reclaimPolicy` - when `PV` is freed from `PVC` what should be done with created `PersistentVolume`

As mentioned previously, one of `Delete` or `Retain` available.

## Expandable Volumes

> From `k8s` 1.11 one can __expand volumes dynamically__

This happens, when we change storage requirements in our `PVC` and `apply` new config.

> Easiest way to use this feature is to use one of internal cloud providers

__All we have to do is set `.allowVolumeExpansion: true` in our `StorageClass` definition__

Below is a list of `providers` supporting expandable volumes:

![](./images/expandable-volumes.png)

# Storage summary

So, given all of the above, a rough guideline one could follow?

> __Decide how your data should be shared__

Answer this question first:
- Shared between containers or pods?

If it is shared between containers answer this question:
- Should data be preserved after pod termination?

__If yes, use `ephemeral volumes` to simply exchange data between applications (`LAMP` example above)__

__If it is shared between pods or should be preserved use `PersistentVolumes`__

## Chose `PersistentVolume`, what now?

Another question to help you:

- Do I need dynamic provisioning (e.g. Hard to know beforehand how much storage I will need?)

If not create the following:
- `PersistentVolume` `.yml` config (defines how to create `volume`)
- `PersistentVolumeClaim` `.yml` config (specifies how a `volume` is requested)
- `MyApplication` `.yml` config (your `workload resource`, __avoid bare pods__)

## I need `dynamic` provisioning, what now?

> __Use this for large scale apps where "by-hand" provisioning is infeasible__

This might happen due to a few reasons:
- A lot of pods requesting a lot of storage
- We cannot see beforehand how many pods will run

> Prefer cloud storage providers in this case, __as one might run out of local storage for large deployments__

In this case, to the steps outlined above one should add another `.yml` config file:
- `StorageClass` `.yml` config (acts as a template for giving out `PersistentVolume`s to pods in need)

## Other options?

Yes, one can also use aforementioned `expandable volumes` if:
- you know how many pods __at most__ will run at any given time
- you are not sure how much storage will be needed for each pod

# StatefulSets
   
Previously shown `workload`s are used for __stateless applications__ (e.g. the ones not writing to external storage).

In a nutshell, StatefulSets are a type of workload resource (so it manages pods) that adds a storage to the pod. That way, everything you run in the pod will be stored somewhere, and next time you run the same pod, the same data will still be there, hence the name 'Stateful'. This is useful for example, if you want to run a pod that depends on a database where you want to store the data in a SQL database.

## Limitations

- `Storage` must be provisioned by admin (or by `StorageClass` dynamically)
- __Deleting will not delete `Storage`__ (preserving storage > automatic purging)
- __Headless `Service`__ is used for network identity of `POD`s __and we have to create it__

## Components

Let's see `.yaml` definitions necessary for `StatefulSet`:

```
apiVersion: v1
kind: Service
metadata:
  name: nginx
  labels:
    app: nginx
spec:
  ports:
  - port: 80
    name: web
  clusterIP: None
  selector:
    app: nginx
---
apiVersion: apps/v1
kind: StatefulSet
metadata:
  name: web
spec:
  serviceName: "nginx"
  replicas: 2
  selector:
    matchLabels:
      app: nginx
  template:
    metadata:
      labels:
        app: nginx
    spec:
      containers:
      - name: nginx
        image: k8s.gcr.io/nginx-slim:0.8
        ports:
        - containerPort: 80
          name: web
        volumeMounts:
        - name: www
          mountPath: /usr/share/nginx/html
  volumeClaimTemplates:
  - metadata:
      name: www
    spec:
      accessModes: [ "ReadWriteOnce" ]
      resources:
        requests:
          storage: 1Gi


- A Headless Service, named `nginx`, is used to control the network domain.
- The `StatefulSet`, named web, has a Spec that indicates that 3 replicas of the `nginx` container will be launched in unique Pods.
- The `volumeClaimTemplates` will provide stable storage using `PersistentVolumes` provisioned by a `PersistentVolume` Provisioner.

## Try it out


1. Create a .yaml file with the configuration above for creating a PersistentVolume resource
2. `apply` it using the right `kubectl`
3. Go to the dashboard and observe the PVCs

In [15]:
!kubectl

service/nginx created
statefulset.apps/web created


The Dashboard should present something like this:
<p align=center><img src=images/PVC.png></p>

Observe that the volumes are already Bound

# Challenges

## Mandatory

- What is and how [Mount Propagation](https://kubernetes.io/docs/concepts/storage/volumes/#mount-propagation) works for volumes?
- What is [Storage Object in Use Protection](https://kubernetes.io/docs/concepts/storage/persistent-volumes/#storage-object-in-use-protection)?
- What is [Volume Binding Mode](https://kubernetes.io/docs/concepts/storage/storage-classes/#volume-binding-mode) for `StorageClasses`?
- What [Allowed Topologies](https://kubernetes.io/docs/concepts/storage/storage-classes/#allowed-topologies) mean for `StorageClasses`?
- Check out [Update Strategies](https://kubernetes.io/docs/concepts/workloads/controllers/statefulset/#update-strategies) for `StatefulSet`s

## Additional

- What is `DefaultStorageClass` admission plugin? How it changes behavior for __unspecified `StorageClass`__ in case of `PV` or `PVC`?
- Check out `Volume Snapshots` feature of `PersistentVolume`s [here](https://kubernetes.io/docs/concepts/storage/volume-snapshots/)
- How to create __non-empty `PersistentVolume`s?__ One can find answer in [Volume populators](https://kubernetes.io/docs/concepts/storage/persistent-volumes/#volume-populators-and-data-sources) section.
- Check how to monitor health of your `Volume`s [here](https://kubernetes.io/docs/concepts/storage/volume-health-monitoring/)